# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

We will need to convert all features into a int? No objects. Missing values??? info() suggested

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

In [5]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   payment_history         1599 non-null   float64
 1   location_parameter      1599 non-null   float64
 2   stem_degree_score       1599 non-null   float64
 3   gpa_ranking             1599 non-null   float64
 4   alumni_success          1599 non-null   float64
 5   study_major_code        1599 non-null   float64
 6   time_to_completion      1599 non-null   float64
 7   finance_workshop_score  1599 non-null   float64
 8   cohort_ranking          1599 non-null   float64
 9   total_loan_score        1599 non-null   float64
 10  financial_aid_score     1599 non-null   float64
 11  credit_ranking          1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"].values

# Display a sample of y
y[:5]

array([0, 0, 0, 1, 0], dtype=int64)

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")

# Review the features DataFrame
X.head(5)

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit and transform the training data, and only transform the testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [11]:
# Define the number of inputs (features) to the model
input_nodes = len(X_train_scaled[0])

# Review the number of features
print(input_nodes)

11


In [12]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_layer_neurons = 1

In [13]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_nodes, activation="relu")
)

# Add the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(tf.keras.layers.Dense(units=output_layer_neurons, activation="sigmoid"))

c:\Users\gisi_\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [15]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [16]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4602 - loss: 0.8319
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4670 - loss: 0.7459
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4861 - loss: 0.7238 
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5184 - loss: 0.6936 
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5677 - loss: 0.6678 
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5834 - loss: 0.6691 
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6391 - loss: 0.6519 
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6428 - loss: 0.6510 
Epoch 9/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6947 - loss: 0.6365 
Epoch 10/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7039 - loss: 0.6349 
Epoch 11/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7223 - loss: 0.6167
Epoch 12/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [17]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display evaluation results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


13/13 - 0s - 22ms/step - accuracy: 0.7425 - loss: 0.5134
Loss: 0.5133925080299377, Accuracy: 0.7425000071525574


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [18]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn.save(file_path)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [19]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [20]:
# Make predictions with the test data
# Display a sample of the predictions
predictions = nn_imported.predict(X_test_scaled,verbose=2)
predictions


13/13 - 0s - 12ms/step


array([[0.6904966 ],
       [0.3879743 ],
       [0.7969106 ],
       [0.7125543 ],
       [0.95285904],
       [0.894609  ],
       [0.9169649 ],
       [0.1425834 ],
       [0.4590674 ],
       [0.35017917],
       [0.88798016],
       [0.29521456],
       [0.50284743],
       [0.9256822 ],
       [0.6517423 ],
       [0.4170479 ],
       [0.9122735 ],
       [0.33960745],
       [0.5934463 ],
       [0.3615649 ],
       [0.65353334],
       [0.8285413 ],
       [0.286232  ],
       [0.8608812 ],
       [0.29098102],
       [0.94542915],
       [0.89317954],
       [0.6138164 ],
       [0.19047247],
       [0.11910802],
       [0.5344714 ],
       [0.9297285 ],
       [0.2189082 ],
       [0.9333298 ],
       [0.10020617],
       [0.4897737 ],
       [0.26587105],
       [0.34634852],
       [0.93974996],
       [0.20637405],
       [0.90706474],
       [0.09321085],
       [0.15239221],
       [0.9410862 ],
       [0.10358874],
       [0.8240598 ],
       [0.17450146],
       [0.356

In [21]:
# Save the predictions to a DataFrame and round the predictions to binary results
df_predictions = pd.DataFrame(columns=["predictions"], data=predictions)
df_predictions["predictions"] = round(df_predictions["predictions"], 0) 
df_predictions

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [22]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, df_predictions["predictions"]))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       188
           1       0.78      0.72      0.75       212

    accuracy                           0.74       400
   macro avg       0.74      0.74      0.74       400
weighted avg       0.75      0.74      0.74       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

1.= I think I would collect data such as:
    
    -Student’s FICO score
    -Loan amount
    -Interest rates
    -Repayment period
    -Employment status
    -Criminal record
    -other demographic info, like age, income level, education.
This data is relevant because it directly influences a student’s ability to repay loans and make informed financial decisions.

2.= I believe I would use collaborative filtering. I could use data from similar users to make recommendations (e.g., students with similar credit scores). Item-based Collaborative Filtering take the similarities between items themselves. For example, if certain loan packages have been chosen by multiple users with similar profiles, these loan options might be recommended to new users with similar profiles. The data I chose would be suitable because this method works by finding patterns in user behaviors or preferences. The basic idea is that if a group of users has similar preferences or behaviors, they are likely to share other preferences as well. 

3.= more than two:

- Privacy: Student financial information is sensitive and needs strict data security measures.
- Bias in recommendations: If historical data contains biased decisions (e.g., favoring students with higher incomes), the recommendation system might inherit these biases.
- Frequency of model retraining: The accuracy and reliability of our predictive model can degrade over time as new data is generated. This phenomenon is known as model drift, where the statistical properties of the target variable (loan repayment behavior) change over time, rendering the model less effective.
- Robustness: Refers to the model’s ability to maintain reliable performance under various conditions, including noisy data, outliers, changes in user behavior, and external factors. One of the biggest risks in building predictive models is overfitting, where the model learns the noise or peculiarities of the training data rather than general patterns. Real-world datasets often contain outliers or corrupted data (e.g., incorrectly entered loan amounts, or extreme credit rankings that don't reflect reality). If the model is not robust to these outliers, it may make inaccurate predictions or recommendations. Over time, the underlying data distribution may change due to factors such as economic trends, government regulations, or changes in student behavior. A robust model should handle such drifts and continue to perform well over time.

